In [1]:
from typing import List, Tuple
import pickle
import os
import time

import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import evaluate
metric = evaluate.load("seqeval")

import sys
sys.path.insert(0, 'D:\\progamming\\va\\truecase\\ru-punctuation-truecase\\src')
from process_text import clean_text, clean_text_3times

# ========== Data global variables ==========
PATH_TO_DATA = "../data"

# ========== Model global variables ==========
MODEL_NAME = "DeepPavlov/rubert-base-cased-conversational"
# "DeepPavlov/rubert-base-cased-conversational" -> rubert-base-cased-conversational
SHORT_MODEL_NAME = MODEL_NAME.split('/')[1] if '/' in MODEL_NAME else MODEL_NAME
MODEL_MAX_LENGTH = 512

In [2]:
def encode_tags(tags, tag2id, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels


def compute_metrics(eval_preds, label_names):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
text = ['кто', 'такой', 'иван', 'иванов']
text_tags = ['U', 'O', 'U', 'U']
label2id = {'O': 0, 'U': 1}
id2label = {0: 'O', 1: 'U'}

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=MODEL_MAX_LENGTH)
inputs = tokenizer(text_words, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
inputs

In [ ]:
text = [['один', 'раз', 'в', 'жизни', 'я', 'делаю', 'хорошее', 'дело', 'и', 'оно', 'бесполезно'],
        ['кто', 'такой', 'иван', 'иванов']]

# text = ['один', 'раз', 'в', 'жизни', 'я', 'делаю', 'хорошее', 'дело', 'и', 'оно', 'бесполезно']

text_tags = [['U', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U', 'O', 'O'], 
             ['U', 'O', 'U', 'U']]

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=MODEL_MAX_LENGTH)
inputs = tokenizer(text, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True, return_tensors="pt")

test_labels = encode_tags(text_tags, label2id, inputs)
inputs.pop("offset_mapping")

test_labels

In [5]:
def split_texts_into_words(texts: List[str]):
    texts_words = []
    for text in texts:
        texts_words.append(text.split(' '))
    return texts_words


def tokenize_texts(texts_words: List[List[str]], tokenizer):
    inputs = tokenizer(texts_words, is_split_into_words=True, padding=True, truncation=True, return_tensors="pt")
    return inputs


def predict_token_classification(inputs, token_classification_model):
    with torch.no_grad():
        logits = token_classification_model(**inputs).logits

    predictions = torch.argmax(logits, dim=2)
    return predictions


def restore_capitalization(texts: List[str], tokenizer, token_classification_model) -> List[str]:
    texts_words = split_texts_into_words(texts)
    inputs = tokenize_texts(texts_words, tokenizer)
    predictions = predict_token_classification(inputs, token_classification_model)

    truecase_texts = []
    for text, model_input, predict in zip(texts_words, inputs.encodings, predictions):
        predicted_token_class = [model.config.id2label[t.item()] for t in predict]
        
        word_class = {}
        for word_id, token_class in zip(model_input.word_ids, predicted_token_class):
            if (word_id != None) and (not word_id in word_class):
                word_class[word_id] = token_class
        
        truecase_words = []
        for i, word in enumerate(text):
            is_upper = word_class[i] == 'U'
            if is_upper:
                truecase_word = word.capitalize()
            else:
                truecase_word = word
            
            truecase_words.append(truecase_word)

        truecase_texts.append(' '.join(truecase_words))
    return truecase_texts

In [6]:
test_queries = [
    "меня зовут сергей а как тебя",
    "подскажи пожалуйста сегодня вторник или среда",
    "закрой за мной дверь я ухожу",
    "в каком году родилась алла пугачёва",
    "когда родилась алла пугачёва",
    "когда родилась пугачёва",
    "скажи год рождения пугачевой",
    "год рождения аллы пугачевой",
    "год рождения пугачевой",
    "день рождения пугачевой",
    "когда день рождения у аллы пугачевой",
    "в каком году родилась алла пугачева",
    "год рождения пугачевой",
    "иван пятый это кто",
    "кто такой иван пятый",
    "кто такой иван пятый",
    "что такое лыжи",
    "швеция столица",
    "столица швеции",
    "какая столица швеции",
    "как называется столица у швеции",
    "расскажи когда родился пушкин",
    "когда дата рождения пушкина",
    "скажи дату рождения пушкина",
    "сколько прожил пушкин",
    "когда родился пушкин",
    "когда родился александр сергеевич пушкин",
    "когда родился александр пушкин",
    "кто такой лев николаевич толстой",
    "кто такой лев толстой",
    "сколько лет путину",
    "возраст путина",
    "какой возраст у владимира путина",
    "какой возраст у путина",
    "какой возраст у владимира владимировича путина",
    "праздник благовещение какого числа",
    "что такое благовещение пресвятой богородицы",
    "кому принадлежит компания газпром",
    "кто директор газпрома",
    "кто является директором компании газпром",
    "сколько лет москве",
    "сколько лет городу москва",
    "в каком году умер брежнев",
    "в каком году скончался леонид брежнев"
]

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=MODEL_MAX_LENGTH)

checkpoint = 'D:/progamming/va/truecase/ru-punctuation-truecase/src/results/rubert-base-cased-conversational-512-tatoeba_dataset/20-53-40/checkpoint-31505/'
# checkpoint = "D:/progamming/va/truecase/ru-punctuation-truecase/src/results/checkpoint-18903/"
model = AutoModelForTokenClassification.from_pretrained(checkpoint)

inference_results = restore_capitalization(test_queries, tokenizer, model)
for query, result in zip(test_queries, inference_results):
    print(f'Query   : {query}')
    print(f'Combined: {result.strip()}\n')

Query   : меня зовут сергей а как тебя
Combined: Меня зовут Сергей А как тебя

Query   : подскажи пожалуйста сегодня вторник или среда
Combined: Подскажи пожалуйста сегодня вторник или среда

Query   : закрой за мной дверь я ухожу
Combined: Закрой за мной дверь я ухожу

Query   : в каком году родилась алла пугачёва
Combined: В каком году родилась Алла Пугачёва

Query   : когда родилась алла пугачёва
Combined: Когда родилась алла пугачёва

Query   : когда родилась пугачёва
Combined: Когда родилась Пугачёва

Query   : скажи год рождения пугачевой
Combined: Скажи год рождения Пугачевой

Query   : год рождения аллы пугачевой
Combined: Год рождения Аллы Пугачевой

Query   : год рождения пугачевой
Combined: Год рождения Пугачевой

Query   : день рождения пугачевой
Combined: День рождения Пугачевой

Query   : когда день рождения у аллы пугачевой
Combined: Когда день рождения у Аллы Пугачевой

Query   : в каком году родилась алла пугачева
Combined: В каком году родилась Алла Пугачева

Query   

In [ ]:
checkpoint = "D:/progamming/va/truecase/ru-punctuation-truecase/src/results/checkpoint-18903/"
model = AutoModelForTokenClassification.from_pretrained(checkpoint)

with torch.no_grad():
    logits = model(**inputs).logits

predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[1]]

print(predictions)
print(predicted_token_class)

In [ ]:
truecase_texts = []
for sentence, model_input, predict in zip(text, inputs.encodings, predictions):
    predicted_token_class = [model.config.id2label[t.item()] for t in predict]
    
    word_class = {}
    for word_id, token_class in zip(model_input.word_ids, predicted_token_class):
        if (word_id != None) and (not word_id in word_class):
            word_class[word_id] = token_class
    
    truecase_words = []
    for i, word in enumerate(sentence):
        is_upper = word_class[i] == 'U'
        if is_upper:
            truecase_word = word.capitalize()
        else:
            truecase_word = word
        
        truecase_words.append(truecase_word)

    truecase_texts.append(' '.join(truecase_words))
truecase_texts

In [ ]:
sample_input = inputs[1]
sample_predict = predictions[1]
sample_input.word_ids

In [ ]:
sample_input

In [ ]:
word_class = {}
for word_id, token_class in zip(sample_input.word_ids, sample_predict):
    if (word_id != None) and (not word_id in word_class):
        word_class[word_id] = token_class.item()

truecase_words = []
for i, word in enumerate(text[1]):
    is_upper = word_class[i]
    if is_upper:
        truecase_word = word.capitalize()
    else:
        truecase_word = word
    
    truecase_words.append(truecase_word)

' '.join(truecase_text)

In [ ]:
truecase_text = []
for i in range(len(text[1])):
    is_upper = word_class[i]
    if is_upper:
        truecase_word = text[1][i].capitalize()
    else:
        truecase_word = text[1][i]
    
    truecase_text.append(truecase_word)

' '.join(truecase_text)